## Utils

### Imports

In [1]:
import requests
from environments.alfworld.common.environment import ALFWorldEnv

import os
from langchain_openai import ChatOpenAI
from planning_library.strategies import ADaPTStrategy
from planning_library.strategies.adapt.components import ADaPTPlanner, ADaPTExecutor
from planning_library.action_executors import GymnasiumActionExecutor
from textwrap import dedent

%load_ext autoreload
%autoreload 2

### Logging

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ALFWorld + ADaPT (new)"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

## Environment

In [3]:
os.environ["ALFWORLD_DATA"] = "utils/data"

if not "base_config.yaml" in os.listdir("utils"):
    response = requests.get("https://raw.githubusercontent.com/alfworld/alfworld/master/configs/base_config.yaml")
    
    with open("utils/base_config.yaml", "wb") as f:
        f.write(response.content)

env = ALFWorldEnv(config_path="utils/base_config.yaml")

Initializing AlfredTWEnv...


100%|██████████| 8810/8810 [00:03<00:00, 2791.93it/s]

Overall we have 3553 games in split=train
Training with 3553 games


In [4]:
obs, info = env.reset(options={"next_episode": True})
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.


## Strategy

### Hyperparameters

In [5]:
# ADaPT hyperparameters
max_depth = 3  # maximum decomposition depth
executor_max_iterations = 2  # maximum number of iterations for the executor; when reached, the loop will exit
planner_max_iterations = 20  # maximum number of iterations for the planner; when reached, the loop will exit

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

### Executor

Default Executor is powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).

In [6]:
action_executor = GymnasiumActionExecutor(env)
executor = ADaPTExecutor.create_simple_strategy(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                                tools=env.tools,
                                             parser_name="openai-tools", 
                                                action_executor=action_executor,
                                                user_message=dedent("""
            You are ALFRED, an intelligent agent navigating in a household.

            {inputs}"""),      max_iterations=executor_max_iterations,
                                                return_intermediate_steps=True,
                                                return_finish_log=True)

### Planner

Default Planner is powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).

In [7]:
planner = ADaPTPlanner.create_simple_strategy(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                             parser_name="openai-tools", 
                                                user_message=dedent("""
            Here is the essence of the original task:

            {inputs}
            """),
            executor_parser_name="openai-tools")

## Defining strategy

In [8]:
adapt = ADaPTStrategy(
    action_executor=action_executor,
    executor=executor,
    planner=planner,
    max_depth=max_depth,
    return_intermediate_steps=True,
    return_finish_log=True,
)

In [9]:
adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...


> Finished chain.


> Entering new SimpleStrategy chain...
I have devised a step-by-step plan with an "and" aggregation logic to successfully solve the task:

1. Head to the shelf 1 and observe the items on it.
2. Put the pencil on the shelf 1.

This plan ensures that both subtasks need to be completed successfully in order to consider the task as solved.

> Finished chain.


> Entering new SimpleStrategy chain...
Task completed.

> Finished chain.


> Entering new SimpleStrategy chain...
It seems like I couldn't put the pencil on shelf 1. Let me try again.

> Finished chain.


> Entering new SimpleStrategy chain...
I have constructed a step-by-step plan with "and" aggregation logic to successfully solve the task of putting the pencil on shelf 1. The plan is as follows:

1. Put the pencil on shelf 1.

This plan will ensure that each subtask is executed sequentially, and the task will only be considered s

{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[(OpenAIToolAgentAction(tool='goto', tool_input={'receptable_type': 'shelf', 'receptable_id': 1}, log="\nInvoking: `goto` with `{'receptable_type': 'shelf', 'receptable_id': 1}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_5YrfsRApVcLBJyFIQMA8S8Cp', 'function': {'arguments': '{"receptable_type":"shelf","receptable_id":1}', 'name': 'goto'}, 'type': 'function'}]})], tool_call_id='call_5YrfsRApVcLBJyFIQMA8S8Cp'),
    {'observation': 'You arrive at loc 9. On the shelf 1, you see a keychain 2, and a pen 1.',
     'reward': 0,
     'terminated': False,
     'truncated': False,
     'info': {}